# Exercise Guide #1: Chat Client

Welcome! Please connect your computer to the wifi and open a terminal. This might be called Terminal.app on a mac, Terminal on Ubuntu, or cmd on Windows. In Windows, go to search and look for cmd.exe.

Type the following into your terminal:

```
nc ipaddress 1234
```
and hit enter, or if you installed netcat from nmap on Windows, type
```
ncat ipaddress 1234
```
If you haven't installed netcat yet on Windows, you can get it from ?

## Using a REPL

Open another terminal, keeping the old one visible.

Next you'll want to type "python3" in the terminal and hit enter. Depending on how you installed Python 3 on Windows you may need to change directories to the python installation folder if this doesn't work. Here's what this looks like on my machine:
```
$ python3
Python 3.6.1 (default, Apr  4 2017, 09:40:21)
[GCC 4.2.1 Compatible Apple LLVM 8.1.0 (clang-802.0.38)] on darwin
Type "help", "copyright", "credits" or "license" for more information.
>>> s = "this creates a string create a variable s that refers to it"
```

You can type Python code here and hit enter and you'll see the result below.

# Exercise: Square 3294

If you're not sure how to square a number in Python, you can multiply it by itself instead.
Put a green sticky note up once you know the answer.

# Exercise: Connecting to the chat server with Python
Type exactly this code in at the Python prompt:
```
>>> import socket
>>> s = socket.socket()
>>> s.connect(('TODO with internet - replace with my ip', 1234))
>>> s.send(b'hello!')  # you can type something other than hello here
```

# Project: Write a chat client¶

Let's write a program similar to nc that we've been using to chat. It should

* connect to the server
* ask for a message from the user
* send that message to the server
* receive data from the server
* display that data
* repeat most of that

Here are some piece of code that will be useful:

    socket.socket() - creates a new socket
    socket.connect(('123.123.123.123', 1234)) - tells OS what address we want this socket to send data to. Be careful to use a tuple!
    socket.recv(1234) - receive up to this number of bytes
    input() - get message from the user
    socket.send()
    while True: - loop forever
    .encode() method on strings
    
The official Python socket docs may be useful too: https://docs.python.org/3/library/socket.html

## Extensions to this project for early finishers

There's a major issue with the simple version of this chat client: it doesn't check for messages until the user finishes typing a message! This seems like a fundemental limitation of .send() and .recv() as we've learned them: these are both blocking calls.

It's typical for networked applications to need to deal with this kind of concurrency.
Solutions:

### use separate programs
Imagine one program which only read from the chat server, and another in a separate terminal that allowed the user to type! We already have this somewhat: our nc program is constantly printing, so we could use that plus this chat client.

### use threads
Let's try the previous idea but in the same program! Threading in Python looks something like this:
```python
import threading

def thing1():
    while True:
        print('hi')
        time.sleep(3)
        

t = threading.Thread(target=thing1)
t.daemon = True
t.start()  # this starts the other thread running, but the main thread continues to the next line

while True:  # this code is being run on the main thread as normal
    print(input().upper())
```

### use nonblocking socket
You can change the api that a socket will use by making it a nonblocking socket!
Now at least when you try to read bytes off of the socket you won't be stuck until some happen, although
you will need to catch an exception if there aren't any ready and check again soon.

```python
s = socket.socket()
s.connect(('ipaddress', 1234))
s.setblocking(False)
s.recv(1000)
```

### use select
On unix-like operating systems, sockets and the stdin stream are both file desciptors. `select` provides a way to block on multiple things at once, and whichever ones are ready first are returned. A select loop might look like this:

```python
while True:
    ready_to_read, _, _ = select.select([sys.stdin, s], [], [])  # blocks on both the socket and stdin
    for stream in ready_to_read:
        if stream is sys.stdin:
            # read from sys.stdin
        else:  # stream is the socket
            # read from the socket
```

# Project: let's extend our chat clients!

If you didn't finish one before, you can start by pasting this code into a Python file:

```python
import socket

def main():
    s = socket.socket()
    s.connect(('123.456.789.012', 1234))
    while True:
        message_to_send = input('msg: ')
        if message_to_send:
            s.send(message_to_send.encode('ascii'))
        print(s.recv(1000))

main()
```